(example-optuna)=

 Example: Model selection with optuna
=========================================

Motivation
----------

We know that model selection and/or hyperparameter optimization (HPO) can
have massive impacts on the prediction quality in regular Machine
Learning. Yet, it seems that model selection and hyperparameter
optimization are  of substantial importance for CATE estimation with
MetaLearners, too, see e.g. [Machlanski et. al](https://arxiv.org/abs/2303.01412>).

However, model selection and HPO for MetaLearners look quite different from what we're used to from e.g. simple supervised learning problems. Concretely,

* In terms of a MetaLearners's option space, there are several levels
  to optimize for:

  1. The MetaLearner architecture, e.g. R-Learner vs DR-Learner
  2. The model to choose per base estimator of said MetaLearner architecture, e.g. ``LogisticRegression`` vs ``LGBMClassifier``
  3. The model hyperparameters per base model

*  On a conceptual level, it's not clear how to measure model quality
   for MetaLearners. As a proxy for the underlying quantity of
   interest one might look into base model performance, the R-Loss of
   the CATE estimates or some more elaborate approaches alluded to by
   [Machlanski et. al](https://arxiv.org/abs/2303.01412).

We think that HPO can be divided into two camps:

* Exploration of (hyperparameter, metric evaluation) pairs where the
  pairs do not influence each other (e.g. grid search, random search)

* Exploration of (hyperparameter, metric evaluation) pairs where the
  pairs do influence each other (e.g. Bayesian optimization,
  evolutionary algorithms); in other words, there is a feedback-loop between
  sample result and sample

In this example, we will illustrate the latter camp based on an
application of [optuna](https://github.com/optuna/optuna) -- a
popular framework for HPO -- in interplay with ``metalearners``.

Installation
------------

In order to use ``optuna``, we first need to install the package.
We can do so either via conda and conda-forge

```console
$ conda install optuna -c conda-forge
```

or via pip and PyPI

```console
$ pip install optuna
```

Usage
-----

### Loading the data

Just like in our {ref}`example on estimating CATEs with a MetaLearner
<example-basic>`, we will first load some experiment data:

In [1]:
import pandas as pd
from pathlib import Path
from git_root import git_root

df = pd.read_csv(git_root("data/learning_mindset.zip"))
outcome_column = "achievement_score"
treatment_column = "intervention"
feature_columns = [
    column for column in df.columns if column not in [outcome_column, treatment_column]
]
categorical_feature_columns = [
    "ethnicity",
    "gender",
    "frst_in_family",
    "school_urbanicity",
    "schoolid",
]
# Note that explicitly setting the dtype of these features to category
# allows both lightgbm as well as shap plots to
# 1. Operate on features which are not of type int, bool or float
# 2. Correctly interpret categoricals with int values to be
#    interpreted as categoricals, as compared to ordinals/numericals.
for categorical_feature_column in categorical_feature_columns:
    df[categorical_feature_column] = df[categorical_feature_column].astype("category")

Now that we've loaded the experiment data, we can split it up into
train and validation data:

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation, w_train, w_validation = train_test_split(
    df[feature_columns], df[outcome_column], df[treatment_column], test_size=0.25
)

### Optimizing base model hyperparameters

Let's say that we want to work with an
{class}`~metalearners.rlearner.RLearner` and ``LightGBM`` estimators
for base models. We will seek two optimize three hyperparameters of
our base models:

* The number of estimators ``n_estimators`` of our outcome model.
* The max depth ``max_depth`` of our outcome model.
* The number of estimators ``n_estimators`` of our treatment effect
  model.

We can mold this ambition into the following simple script creating an
``optuna`` ``study``:

In [3]:
import optuna
from metalearners.rlearner import r_loss
from metalearners.utils import simplify_output
from metalearners import RLearner
from lightgbm import LGBMRegressor, LGBMClassifier


def objective(trial):

    n_estimators_nuisance = trial.suggest_int("n_estimators_nuisance", 5, 250)
    max_depth_nuisance = trial.suggest_int("max_depth_nuisance", 3, 30)
    n_estimators_treatment = trial.suggest_int("n_estimators_treatment", 5, 100)

    rlearner = RLearner(
        nuisance_model_factory=LGBMRegressor,
        nuisance_model_params={
            "n_estimators": n_estimators_nuisance,
            "max_depth": max_depth_nuisance,
            "verbosity": -1,
        },
        propensity_model_factory=LGBMClassifier,
        propensity_model_params={"n_estimators": 5, "verbosity": -1},
        treatment_model_factory=LGBMRegressor,
        treatment_model_params={
            "n_estimators": n_estimators_treatment,
            "verbosity": -1,
        },
        is_classification=False,
        n_variants=2,
    )

    rlearner.fit(X=X_train, y=y_train, w=w_train)

    return rlearner.evaluate(
        X=X_validation,
        y=y_validation,
        w=w_validation,
        is_oos=True,
    )["r_loss_1_vs_0"]


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

[I 2024-06-06 17:46:41,860] A new study created in memory with name: no-name-8bec8f8a-95e2-4c3d-aa52-ad191a07deee
[I 2024-06-06 17:46:51,040] Trial 0 finished with value: 0.8364624411198488 and parameters: {'n_estimators_nuisance': 165, 'max_depth_nuisance': 29, 'n_estimators_treatment': 91}. Best is trial 0 with value: 0.8364624411198488.
[I 2024-06-06 17:46:53,303] Trial 1 finished with value: 0.7921430800824065 and parameters: {'n_estimators_nuisance': 146, 'max_depth_nuisance': 3, 'n_estimators_treatment': 13}. Best is trial 1 with value: 0.7921430800824065.
[I 2024-06-06 17:47:01,815] Trial 2 finished with value: 0.8322799183534213 and parameters: {'n_estimators_nuisance': 189, 'max_depth_nuisance': 26, 'n_estimators_treatment': 57}. Best is trial 1 with value: 0.7921430800824065.
[I 2024-06-06 17:47:08,605] Trial 3 finished with value: 0.8233201942357238 and parameters: {'n_estimators_nuisance': 155, 'max_depth_nuisance': 26, 'n_estimators_treatment': 30}. Best is trial 1 with va

Note that the metric to be optimized is the R-Loss here. We can obtain
it -- among other metrics -- via the
{class}`~metalearners.rlearner.RLearner`'s
{meth}`~metalearners.rlearner.RLearner.evaluate` method. We can see it
evolve as follows across the 100 trials.

Alternatively, if we'd like to optimize a base model in light of its
individual metric -- in this case an RMSE on the observed outcomes for an
the outcome model -- we can easily do that, too:

In [4]:
from sklearn.metrics import root_mean_squared_error


def objective_individual(trial):

    n_estimators_nuisance = trial.suggest_int("n_estimators_nuisance", 5, 250)
    max_depth_nuisance = trial.suggest_int("max_depth_nuisance", 3, 30)

    rlearner = RLearner(
        nuisance_model_factory=LGBMRegressor,
        nuisance_model_params={
            "n_estimators": n_estimators_nuisance,
            "max_depth": max_depth_nuisance,
            "verbosity": -1,
        },
        propensity_model_factory=LGBMClassifier,
        treatment_model_factory=LGBMRegressor,
        is_classification=False,
        n_variants=2,
    )

    rlearner.fit_nuisance(X=X_train, y=y_train, model_kind="outcome_model", model_ord=0)

    outcome_predictions = rlearner.predict_nuisance(
        X=X_validation, model_kind="outcome_model", model_ord=0, is_oos=True
    )

    return root_mean_squared_error(y_validation, outcome_predictions)


study_individual = optuna.create_study(direction="minimize")
study_individual.optimize(objective_individual, n_trials=100)

[I 2024-06-06 17:52:49,973] A new study created in memory with name: no-name-f44051e8-c810-4eee-a592-7356c606cd95
[I 2024-06-06 17:52:52,977] Trial 0 finished with value: 0.8288007517641651 and parameters: {'n_estimators_nuisance': 81, 'max_depth_nuisance': 24}. Best is trial 0 with value: 0.8288007517641651.
[I 2024-06-06 17:52:55,028] Trial 1 finished with value: 0.8241959001584742 and parameters: {'n_estimators_nuisance': 131, 'max_depth_nuisance': 4}. Best is trial 1 with value: 0.8241959001584742.
[I 2024-06-06 17:53:01,804] Trial 2 finished with value: 0.8459334232683748 and parameters: {'n_estimators_nuisance': 202, 'max_depth_nuisance': 19}. Best is trial 1 with value: 0.8241959001584742.
[I 2024-06-06 17:53:04,731] Trial 3 finished with value: 0.8276873597440942 and parameters: {'n_estimators_nuisance': 75, 'max_depth_nuisance': 29}. Best is trial 1 with value: 0.8241959001584742.
[I 2024-06-06 17:53:12,018] Trial 4 finished with value: 0.848063716441658 and parameters: {'n_es

### Optimizing over architectures

``optuna``'s flexibility allows for not only the search over classical
hyperparameters of a given estimator but also to iterate over the
choice of base estimator architectures. Pushing it a step further, one
can even optimize over the space of MetaLearner architectures.

In the following example we will attempt to optimize over the
following search space:

1. MetaLearner: R-Learner vs DR-Learner
2. Nuisance model: ``LGBMRegressor`` vs ``Ridge``
3. Hyperparameter: ``n_estimators`` if ``LGBMRegressor`` and ``alpha``
   if ``Ridge``

Note that the choice of the base learner in the second step should be
conditioned on the choice in the first step. In other words, we do not
want to update our belief system on outcome learners for the R-Learner
by observing outcome learner for the DR-Learner. The same idea applies
to the interplay between steps two and three. This conditioning
becomes apparent in the source code below via the underscores,
e.g. ``nuisance_r``, which is only sampled (and thereby updated) if we
are using an R-Learner and and ``nuisance_dr``, which is only sampled
(and thereby updated) if we are using a DR-Learner.

In [5]:
import optuna
from metalearners.utils import metalearner_factory, simplify_output
from sklearn.linear_model import Ridge
from metalearners.rlearner import r_loss


# Arbitrary models for R-Loss
outcome_estimates = LGBMRegressor().fit(X_train, y_train).predict(X_validation)
propensity_scores = LGBMClassifier().fit(X_train, w_train).predict(X_validation)


def objective_overall(trial):

    ### SAMPLING

    # Highest level of granularity: we sample the MetaLearner architecture.
    architecture = trial.suggest_categorical("architecture", ["R", "DR"])

    # We distinguish cases because we do not want a DR-Learner run to influence
    # the optimizing process of R-Learner-related parameters.
    if architecture == "R":

        # Second level of granularity: we sample the nuisance base model.
        # Note that this is conditioned on using the R-Learner.
        nuisance_r = trial.suggest_categorical("nuisance_r", ["LGBMRegressor", "Ridge"])
        nuisance_dr = None
        nuisance_dr_log_reg_alpha = None
        nuisance_dr_lgbm_n_estimators = None

        if nuisance_r == "LGBMRegressor":

            # Lowest level of granularity: we sample the nuisance base model hyperparameters.
            nuisance_r_lgbm_n_estimators = trial.suggest_int(
                "nuisance_r_lgbm_n_estimators", 5, 250
            )
            nuisance_r_lin_reg_alpha = None

            nuisance_params = {
                "n_estimators": nuisance_r_lgbm_n_estimators,
                "verbose": -1,
            }
        else:
            nuisance_r_lin_reg_alpha = trial.suggest_float(
                "nuisance_r_lin_reg_alpha", 0, 10
            )
            nuisance_r_lgbm_n_estimators = None

            nuisance_params = {"alpha": nuisance_r_lin_reg_alpha}

    else:
        nuisance_dr = trial.suggest_categorical(
            "nuisance_dr", ["LGBMRegressor", "Ridge"]
        )
        nuisance_r = None
        nuisance_r_lin_reg_alpha = None
        nuisance_r_lgbm_n_estimators = None

        if nuisance_dr == "LGBMRegressor":

            # Lowest level of granularity: we sample the nuisance base model hyperparameters.
            nuisance_dr_lgbm_n_estimators = trial.suggest_int(
                "nuisance_dr_lgbm_n_estimators", 5, 250
            )
            nuisance_dr_lin_reg_alpha = None

            nuisance_params = {
                "n_estimators": nuisance_dr_lgbm_n_estimators,
                "verbose": -1,
            }

        else:
            nuisance_dr_lin_reg_alpha = trial.suggest_float(
                "nuisance_dr_lin_reg_alpha", 0, 10
            )
            nuisance_dr_lgbm_n_estimators = None

            nuisance_params = {"alpha": nuisance_dr_lin_reg_alpha}

    ### LEARNING

    _metalearner_factory = metalearner_factory(architecture)
    # We know that only one of them is not None, therefore we can use or.
    nuisance_model_type = nuisance_r or nuisance_dr
    metalearner = _metalearner_factory(
        nuisance_model_factory=(
            LGBMRegressor if nuisance_model_type == "LGBMRegressor" else Ridge
        ),
        nuisance_model_params=nuisance_params,
        propensity_model_factory=LGBMClassifier,
        propensity_model_params={"n_estimators": 5, "max_depth": 5, "verbose": -1},
        treatment_model_factory=LGBMRegressor,
        treatment_model_params={"n_estimators": 5, "max_depth": 5, "verbose": -1},
        is_classification=False,
        n_variants=2,
    )
    metalearner.fit(X_train, y_train, w_train)

    ### EVALUATING

    cate_estimates = simplify_output(metalearner.predict(X_validation, is_oos=True))

    return r_loss(
        cate_estimates=cate_estimates,
        outcome_estimates=outcome_estimates,
        propensity_scores=propensity_scores,
        outcomes=y_validation,
        treatments=w_validation,
    )


study_overall = optuna.create_study(direction="minimize")
study_overall.optimize(objective_overall, n_trials=100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000248 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 496
[LightGBM] [Info] Number of data points in the train set: 7793, number of used features: 11
[LightGBM] [Info] Start training from score -0.002111
[LightGBM] [Info] Number of positive: 2565, number of negative: 5228
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000253 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 496
[LightGBM] [Info] Number of data points in the train set: 7793, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.329142 -> initscore=-0.712070
[LightGBM] [Info] Start training from score -0.712070


[I 2024-06-06 17:56:01,747] A new study created in memory with name: no-name-06cb1974-6a0c-4cdf-9a2d-6a8a7c221df7
[I 2024-06-06 17:56:02,233] Trial 0 finished with value: 0.8169250800494238 and parameters: {'architecture': 'R', 'nuisance_r': 'Ridge', 'nuisance_r_lin_reg_alpha': 8.28677940660548}. Best is trial 0 with value: 0.8169250800494238.
[I 2024-06-06 17:56:10,695] Trial 1 finished with value: 0.8162437974068578 and parameters: {'architecture': 'R', 'nuisance_r': 'LGBMRegressor', 'nuisance_r_lgbm_n_estimators': 247}. Best is trial 1 with value: 0.8162437974068578.
[I 2024-06-06 17:56:11,179] Trial 2 finished with value: 0.8159429235765234 and parameters: {'architecture': 'DR', 'nuisance_dr': 'Ridge', 'nuisance_dr_lin_reg_alpha': 4.536653702854977}. Best is trial 2 with value: 0.8159429235765234.
[I 2024-06-06 17:56:11,645] Trial 3 finished with value: 0.8166980815884088 and parameters: {'architecture': 'R', 'nuisance_r': 'Ridge', 'nuisance_r_lin_reg_alpha': 0.8092137252574028}. B